In [32]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize


import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# 设置中文字体为宋体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置英文字体为新罗马
plt.rcParams['font.serif'] = ['Times New Roman']
# 字体大小
plt.rcParams['font.size'] = 12  
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False 

In [34]:
df1 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\花菜类.csv')
df1.head()

,日期,销量(千克),销售单价(元/千克),批发价格(元/千克),损耗率_品类,成本加成定价
0,2020-07-01,46.640,12.834951,8.484369,15.51,12.834951
1,2020-07-02,43.943,12.421053,7.832632,15.51,12.421053
2,2020-07-03,42.076,12.000000,7.583333,15.51,12.000000
3,2020-07-04,55.662,12.619048,8.220476,15.51,12.619048
4,2020-07-05,55.474,12.641509,8.442075,15.51,12.641509


In [35]:
predict = pd.read_excel('中间数据集\\未来七天成本预测结果.xlsx')
predict.columns = ['日期', '花菜类', '花叶类', '茄类', '辣椒类', '食用菌', '水生根茎类']
predict

,日期,花菜类,花叶类,茄类,辣椒类,食用菌,水生根茎类
0,未来第1天,7.89,3.16,4.96,4.69,7.63,12.66
1,未来第2天,7.89,3.17,4.93,4.63,7.62,12.58
2,未来第3天,7.89,3.15,4.96,4.65,7.55,12.56
3,未来第4天,7.92,3.10,4.92,4.67,7.56,12.63
4,未来第5天,7.89,3.16,4.96,4.69,7.63,12.66
5,未来第6天,7.89,3.17,4.93,4.63,7.62,12.58
6,未来第7天,7.89,3.15,4.96,4.65,7.55,12.56


In [36]:
range = pd.read_excel('中间数据集\\2_4_各品类对销量与成本加成定价的约束.xlsx')
range.head()

,品类,销量(千克)_下界,销量(千克)_上界,成本加成定价_下界,成本加成定价_上界
0,花菜类,2.493046,115.753382,3.718730,22.908976
1,花叶类,13.864352,471.022934,2.132901,10.954627
2,茄类,-0.344874,57.698732,4.173864,17.451418
3,辣椒类,3.853574,257.681711,2.470155,28.757278
4,食用菌,1.602830,250.875884,6.421218,18.767597


In [37]:
relate = pd.read_excel('结果\\销量-成本加成定价线性关系.xlsx')
relate.columns = ['系数', '花菜类', '花叶类', '茄类', '辣椒类', '食用菌', '水生根茎类']
relate.head()

,系数,花菜类,花叶类,茄类,辣椒类,食用菌,水生根茎类
0,a,-0.032953,-0.004100,-0.065122,-0.031069,-0.031848,-0.047177
1,b,10.550579,6.749194,9.931930,11.426110,14.098714,11.385178


In [38]:
df1_predict = pd.DataFrame({
    '日期': predict['日期'],
    '预测批发价': predict['花菜类'],
    '损耗率_品类': df1['损耗率_品类'],
    }).dropna()
df1_predict

,日期,预测批发价,损耗率_品类
0,未来第1天,7.89,15.51
1,未来第2天,7.89,15.51
2,未来第3天,7.89,15.51
3,未来第4天,7.92,15.51
4,未来第5天,7.89,15.51
5,未来第6天,7.89,15.51
6,未来第7天,7.89,15.51


In [39]:
df1_predict['p/(1-k)'] = df1_predict['预测批发价'] / (1 - (df1_predict['损耗率_品类'] / 100))
df1_predict = df1_predict.round(2)
df1_predict

,日期,预测批发价,损耗率_品类,p/(1-k)
0,未来第1天,7.89,15.51,9.34
1,未来第2天,7.89,15.51,9.34
2,未来第3天,7.89,15.51,9.34
3,未来第4天,7.92,15.51,9.37
4,未来第5天,7.89,15.51,9.34
5,未来第6天,7.89,15.51,9.34
6,未来第7天,7.89,15.51,9.34


目标函数：求最大值

$$f(x, y) = x * (y - \dfrac{p}{1-k})$$

约束条件：

$$y = ax + b$$
$$lower\_i\_1 \leq x \leq upper\_i\_1$$
$$lower\_i\_2 \leq y \leq upper\_i\_2$$
$$x>0, y>0$$

<br><br>

## ***花菜类寻优***

<br><br>

In [40]:
relate

,系数,花菜类,花叶类,茄类,辣椒类,食用菌,水生根茎类
0,a,-0.032953,-0.004100,-0.065122,-0.031069,-0.031848,-0.047177
1,b,10.550579,6.749194,9.931930,11.426110,14.098714,11.385178


**第一天**

In [41]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][0]))

# 定义约束条件
cons = (
    {'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
    {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
    {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
    {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
    )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_1 = result.x[0]
optimal_y_1 = -objective(result.x[0])
max_f_x_1 = -result.fun

**第二天**

In [42]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][1]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_2 = result.x[0]
optimal_y_2 = -objective(result.x[0])
max_f_x_2 = -result.fun

**第三天**

In [43]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][2]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_3 = result.x[0]
optimal_y_3 = -objective(result.x[0])
max_f_x_3 = -result.fun

**第四天**


In [44]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][3]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_4 = result.x[0]
optimal_y_4 = -objective(result.x[0])
max_f_x_4 = -result.fun

**第五天**


In [45]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][4]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_5 = result.x[0]
optimal_y_5 = -objective(result.x[0])
max_f_x_5 = -result.fun

**第六天**


In [46]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][5]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_6 = result.x[0]
optimal_y_6 = -objective(result.x[0])
max_f_x_6 = -result.fun

**第七天**

In [47]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][6]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_7 = result.x[0]
optimal_y_7 = -objective(result.x[0])
max_f_x_7 = -result.fun

<br><br><br>

In [48]:
result_1 = pd.DataFrame({
    '最优销量': [optimal_x_1, optimal_x_2, optimal_x_3, optimal_x_4, optimal_x_5, optimal_x_6, optimal_x_7],
    '最优定价': [optimal_y_1, optimal_y_2, optimal_y_3, optimal_y_4, optimal_y_5, optimal_y_6, optimal_y_7], 
    '最大收益': [max_f_x_1, max_f_x_2, max_f_x_3, max_f_x_4, max_f_x_5, max_f_x_6, max_f_x_7],
    }, index=['第一天', '第二天', '第三天', '第四天', '第五天', '第六天', '第七天'])
result_1['最优补货量'] = result_1['最优销量'] / (1 - df1_predict['损耗率_品类'][0] / 100)

# 写入结果
result_1.to_excel("结果\\2_寻优结果_花菜类.xlsx")
result_1

,最优销量,最优定价,最大收益,最优补货量
第一天,18.368180,11.118061,11.118061,21.740064
第二天,18.368180,11.118061,11.118061,21.740064
第三天,18.368180,11.118061,11.118061,21.740064
第四天,17.913028,10.573844,10.573844,21.201359
第五天,18.368180,11.118061,11.118061,21.740064
第六天,18.368180,11.118061,11.118061,21.740064
第七天,18.368180,11.118061,11.118061,21.740064


# 其他五种蔬菜均如此，我懒得加了